In [1]:
import create_geom_with_pygmsh as cp
import pygmsh
import numpy as np
import scipy.signal

use_cpml = True

def my_sinusoidal(x, amplitude=2000, period=40000, depth=16000):

    z = amplitude * np.sin(2 * np.pi * x / period)
    # taper for both ends
    taper = scipy.signal.tukey(len(z), alpha=0.15)
    z *= taper

    return z + depth # average depth is 16000m (z coordinate from the bottom actually)


def create_sinusoidal_subsurface(n_points, xmin, xmax, depth=16000):
    x_arr = np.linspace(xmin, xmax, n_points)
    z_arr = my_sinusoidal(x_arr, depth=depth)

    return x_arr, z_arr


# Initialize empty geometry using the build in kernel in GMSH
with pygmsh.geo.Geometry() as geom:

    # define global parameters
    H = 20 * 1000  # domain height in meter
    L = 100 * 1000  # domain width in meter

    H_w = 4 * 1000  # water depth in meter
    H_g = (H - H_w)  # subsurface depth in meter

    # element size for both water and subsurface domain
    #lc_w = 0.09375 * 1000
    #lc_g = 0.075 * 1000

    lc_w = 0.15 * 1000.0
    lc_g = 0.25 * 1000.0
    lc_b = min(lc_w, lc_g) # element size at boundary
    H_t = lc_g*2.0 # width of transition layer

    """ node ids without pml layer
    5             6

    4             3
    4t            3t <--- transition layer


    1             2

    """

   # points
    p1 = (0, 0, 0)
    p2 = (L, 0, 0)
    p3 = (L, H_g, 0)
    p4 = (0, H_g, 0)
    p5 = (0, H, 0)
    p6 = (L, H, 0)

    p3t = (L, H_g - H_t, 0)
    p4t = (0, H_g - H_t, 0)

    #
    # create subsurface geometry
    #
    x_arr, z_arr = create_sinusoidal_subsurface(int(L / lc_b), 0, L, depth=16000)
    topo = {"x": x_arr, "z": z_arr}

    x_arr_t, z_arr_t = create_sinusoidal_subsurface(int(L / lc_g), 0, L, depth=16000-H_t)
    topo_t = {"x": x_arr_t, "z": z_arr_t}

    nelm_h_g = int(L / lc_g) + 1
    nelm_h_w = int(L / lc_w) + 1

    # create rectangles
    whole_domain = cp.rectangles(geom)
    whole_domain.add_one_rect(geom, p1, p2, p3t, p4t, lc_g, transfinite=True, mat_tag="M1", nelm_h=nelm_h_g ,topo=topo_t)
    whole_domain.add_one_rect(geom, p4t, p3t, p3, p4, [lc_g, lc_g, lc_w, lc_w], transfinite=False, mat_tag="M1", topo=topo)
    whole_domain.add_one_rect(geom, p4, p3, p6, p5, lc_w, transfinite=True, nelm_h=nelm_h_w, mat_tag="M2")

    # create pml layer
    if use_cpml:
        lc_pml = min(lc_w, lc_g)
        w_pml = lc_pml*10
        whole_domain.add_pml_layers(geom, w_pml, lc_pml, top_pml=False)

    # build up edges and lines
    whole_domain.build_points_edges(geom)

    # force recombine all the surfaces
    import gmsh
    gmsh.option.setNumber("Mesh.RecombineAll", 1)
    gmsh.option.setNumber("Mesh.RecombinationAlgorithm", 2)

    # create and write mesh
    mesh = geom.generate_mesh(dim=2, order=2, verbose=True)
    mesh.write("mesh.xdmf")



/tmp/ipykernel_45823/88217727.py:12: DeprecationWarning: Importing tukey from 'scipy.signal' is deprecated since SciPy 1.1.0 and will raise an error in SciPy 1.13.0. Please use 'scipy.signal.windows.tukey' or the convenience function 'scipy.signal.get_window' instead.
  taper = scipy.signal.tukey(len(z), alpha=0.15)


rectangle id 0 is meshed with nelm_h (bottom,top) = [401, 401]                 and nelm_v (left, right) = None
rectangle id 1 is meshed with nelm_h (bottom,top) = None                 and nelm_v (left, right) = None
rectangle id 2 is meshed with nelm_h (bottom,top) = [667, 667]                 and nelm_v (left, right) = None
bottom rectangle id 0
top rectangle id 2
rectangle id 3 is meshed with nelm_h (bottom,top) = [11, 11]                 and nelm_v (left, right) = None
rectangle id 4 is meshed with nelm_h (bottom,top) = [11, 11]                 and nelm_v (left, right) = None
rectangle id 5 is meshed with nelm_h (bottom,top) = [11, 11]                 and nelm_v (left, right) = None
rectangle id 6 is meshed with nelm_h (bottom,top) = [11, 11]                 and nelm_v (left, right) = None
rectangle id 7 is meshed with nelm_h (bottom,top) = [11, 11]                 and nelm_v (left, right) = None
rectangle id 8 is meshed with nelm_h (bottom,top) = [11, 11]                 and nelm_v

In [2]:
from meshio2spec2d import *

mio2spec = Meshio2Specfem2D(mesh)
mio2spec.write("test")

Time elapsed for nodes:  0.24155712127685547  seconds
Time elapsed for mesh:  0.10703825950622559  seconds
material keys:  ['M1', 'M2']
cell_id_offset:  2552
Time elapsed for material:  0.05549216270446777  seconds
Time elapsed for surf:  1.7438225746154785  seconds
Number of PML_X elements:  1920  ->  1728  ( 192  elements removed)
Number of PML_Y elements:  3200  ->  2800  ( 400  elements removed)
Number of PML_XY elements:  160  ->  158  ( 2  elements removed)
Time elapsed for cpml:  0.5849852561950684  seconds
Time elapsed:  2.7331387996673584  seconds


In [3]:
# write MESH/nummaterial_velocity_file separately

"""
# format:
#(1)domain_id #(2)material_id #(3)rho #(4)vp #(5)vs #(6)Q_k #(7)Q_mu #(8)ani
#
#  where
#     domain_id          : 1=acoustic / 2=elastic / 3=poroelastic
#     material_id        : POSITIVE integer identifier of material block
#     rho                : density
#     vp                 : P-velocity
#     vs                 : S-velocity
#     Q_k                : 9999 = no Q_kappa attenuation
#     Q_mu               : 9999 = no Q_mu attenuation
#     ani                : 0=no anisotropy/ 1,2,.. check with aniso_model.f90
#
# example:
# 2   1 2300 2800 1500 9999.0 9999.0 0
#
# or
#
#(1)domain_id #(2)material_id  tomography elastic  #(3)filename #(4)positive
#
#  where
#     domain_id : 1=acoustic / 2=elastic / 3=poroelastic
#     material_id        : NEGATIVE integer identifier of material block
#     filename           : filename of the tomography file
#     positive           : a positive unique identifier
#
# example:
# 2  -1 tomography elastic tomo.xyz 1
"""

fname = "MESH/nummaterial_velocity_file"

# M1: subsurface
M1_params = "2 1 1850.d0 2400.d0 1200.d0 9999 9999 0"

# M2: water
M2_params = "1 2 1020.d0 1500.d0 0.d0 9999 9999 0"


with open(fname, "w") as f:
    f.write(M1_params + "\n")
    f.write(M2_params + "\n")

In [4]:
# prepare MESH_flat/STATIONS

# prepare STATIONS
#n_rec_x = 49
n_rec_z = 5
l_rec_dx = 2000.0 # in meters
l_rec_dz = 1000.0 # in meters
n_rec_x = int(L / l_rec_dx)
#n_rec_z = int(H_w / l_rec_dz) + 1
x_orig = 2000.0 # in meters from the left bound
z_orig = 16000.0 # in meters from the bottom bound

name_station = "DATA/STATIONS"

with open(name_station, "w") as f:

    for i in range(n_rec_x):

        x = x_orig + i * l_rec_dx
        # at j=0, z is always on the ocean bottom
        #z_bottom = my_sinusoidal(x)
        # interpolate z from x_arr and z_arr
        z_bottom = np.interp(x, x_arr, z_arr)

        l_rec_dz = (20000.0 - z_bottom) / (n_rec_z - 1)# place "n_rec_z" receivers equally between the ocean bottom and the top

        #z_bottom = z_orig

        for j in range(n_rec_z):
            name_sta = "x_" + str(i)
            name_net = "z_" + str(j)

            z = z_bottom + j * l_rec_dz


            # STATIONS
            # format:
            #   #name #network #x #z #elevation #burial_depth
            f.write("{}\t{}\t{}\t{}\t{}\t{}\n".format(name_sta, name_net, x, z, 0.0, 0.0))